In [57]:
import pandas as pd
import numpy as np
import subprocess

In [28]:
def non_negative(number):
    '''helper function, prevent negative regression result'''
    if number < 0: return 0
    else         : return number

In [29]:
def read_table(shortName, site_type): 
    if site_type == 'don':
        return pd.read_csv(don_path.format(shortName), sep = ' ', header=None)
    if site_type == 'acc':
        return pd.read_csv(acc_path.format(shortName), sep = ' ', header=None)

In [69]:
species = pd.read_csv('/home/richard/research/tests/20160102RegressionFromAlexData/191species.csv', header = None)[0]

In [70]:
info = pd.read_csv('/home/richard/research/tests/20160102RegressionFromAlexData/freq_table_real_frame1.csv')
info.index = info.shortName

In [71]:
info = info.ix[species] #pick the species according to the filtered list

In [72]:
don_path = '/storage3/w/alexl/Fa/{}/annot/don.mat'
acc_path = '/storage3/w/alexl/Fa/{}/annot/acc.mat'

In [34]:
don_sample_table = read_table('Aciri1_iso', 'don'); print don_sample_table

   0       1       2       3  4  5       6       7       8       9
0  T  0.1970  0.2415  0.1973  0  1  0.0710  0.2190  0.0638  0.5297
1  C  0.2442  0.2131  0.1292  0  0  0.0739  0.1880  0.0446  0.2119
2  A  0.3358  0.3737  0.1631  0  0  0.6194  0.4825  0.0902  0.1542
3  G  0.2230  0.1717  0.5104  1  0  0.2357  0.1105  0.8014  0.1043


In [35]:
acc_sample_table = read_table('Aciri1_iso', 'acc'); print acc_sample_table

   0       1       2       3       4       5  6  7       8
0  T  0.3224  0.3170  0.3613  0.2059  0.3486  0  0  0.1740
1  C  0.2374  0.2399  0.2515  0.2073  0.5008  0  0  0.1956
2  A  0.2318  0.2351  0.2127  0.3226  0.1193  1  0  0.2660
3  G  0.2084  0.2081  0.1745  0.2642  0.0313  0  1  0.3645


In [36]:
# collect tables

In [37]:
don_tables = []
for shortName in info.shortName:
    try:
        don_tables.append((shortName,read_table(shortName, 'don')))
    except:
        print "problem with",shortName

problem with Antlo1
problem with Entbi1


In [40]:
acc_tables = []
for shortName in info.shortName:
    try:
        acc_tables.append((shortName,read_table(shortName, 'acc')))
    except:
        print "problem with",shortName

problem with Antlo1
problem with Entbi1


In [43]:
# train_model donor
num_rows = 4
num_cols = don_tables[0][1].shape[1]-1

don_model = [[None for j in range(num_cols)] for i in range(num_rows)]
cds_gc = info.cds_gc[map(lambda x: x[0], don_tables)]
for i in range(num_rows):
    for j in range(num_cols):
        don_model[i][j] = np.poly1d(np.polyfit(cds_gc, map(lambda x: x[1].iloc[i,j+1], don_tables), 1))
#         print sample_table.iloc[i,j+1],
#     print

In [44]:
# train_model acceptor
num_rows = 4
num_cols = acc_tables[0][1].shape[1]-1

acc_model = [[None for j in range(num_cols)] for i in range(num_rows)]
cds_gc = info.cds_gc[map(lambda x: x[0], acc_tables)]
for i in range(num_rows):
    for j in range(num_cols):
        acc_model[i][j] = np.poly1d(np.polyfit(cds_gc, map(lambda x: x[1].iloc[i,j+1], acc_tables), 1))
#         print sample_table.iloc[i,j+1],
#     print

In [84]:
# table path
donor_path = '/home/richard/research/tests/20160129ApproxSiteDonorAcceptor/donors/'
donor_path_approx = '/home/richard/research/tests/20160129ApproxSiteDonorAcceptor/approx_donors/'
acceptor_path = '/home/richard/research/tests/20160129ApproxSiteDonorAcceptor/acceptors/'
acceptor_path_approx = '/home/richard/research/tests/20160129ApproxSiteDonorAcceptor/approx_acceptors//'

In [60]:
#create table according to donor model
num_rows = 4
num_cols = don_tables[0][1].shape[1]-1

don_template = don_sample_table.copy()
for gc in range(25,76):
    for i in range(num_rows):
        for j in range(num_cols):
            don_template.loc[i,j+1] = non_negative(don_model[i][j](gc))
            don_template.to_csv('{}{}{}.txt'.format(donor_path_approx, 'don_',gc), sep=" ",index = False, header = None)

In [61]:
#create table according acceptor model
num_rows = 4
num_cols = acc_tables[0][1].shape[1]-1

acc_template = acc_sample_table.copy()
for gc in range(25,76):
    for i in range(num_rows):
        for j in range(num_cols):
            acc_template.loc[i,j+1] = non_negative(acc_model[i][j](gc))
            acc_template.to_csv('{}{}{}.txt'.format(acceptor_path_approx, 'acc_',gc), sep=" ",index = False, header = None)

In [74]:
## copy original donor, acc matrix to folder
# donor
for name in info.shortName:
    try:
        command = 'cp {} {}{}_don_{}.txt'.format(don_path.format(name), donor_path, name, int(round(info.ix[name].genome_gc, 0)))
        subprocess.check_call(command, shell = True)
    except:
        print 'problem with', name

problem with Antlo1
problem with Entbi1


In [87]:
## copy original acceptor, acc matrix to folder
# donor
for name in info.shortName:
    try:
        command = 'cp {} {}{}_acc_{}.txt'.format(acc_path.format(name), acceptor_path, name, int(round(info.ix[name].genome_gc, 0)))
#         print command
        subprocess.check_call(command, shell = True)
    except:
        print 'problem with', name


problem with Antlo1
problem with Entbi1
